In [379]:
import numpy as np 
import pandas as pd 
import re

In [381]:
patients=pd.read_csv("patients.csv")
treatments=pd.read_csv("treatments.csv",na_values=["-"])
treatment_cut=pd.read_csv("treatments_cut.csv",na_values=["-"])
adverse_reactions=pd.read_csv("adverse_reactions.csv")

In [383]:
patients_df=patients.copy()
treatments_df=treatments.copy()
treatment_cut_df=treatment_cut.copy()
adverse_reactions_df=adverse_reactions.copy()

In [385]:
patients_df["assigned_sex"]=patients_df["assigned_sex"].astype("category")
patients_df["birthdate"]=pd.to_datetime(patients_df["birthdate"])

In [407]:
patients_df=patients_df.drop_duplicates(subset=["given_name","surname"],keep="first")
patients_df["zip_code"].fillna("0.0",inplace=True)
patients_df.loc[8,"given_name"]="David"
patients_df[patients_df["patient_id"]==9]
def zip_code(value):
    return str(value).split(".")[0]
def zip_code_length(value):
    if len(str(value))==5:
        return value 
    else:
        return "0"+str(value)
patients_df["zip_code"]=patients_df["zip_code"]=patients_df["zip_code"].apply(zip_code)
patients_df["zip_code"]=patients_df["zip_code"].apply(zip_code_length)
patients_df["zip_code"]=patients_df["zip_code"].astype("int")
# Sample structure for patients_df
# patients_df = pd.DataFrame({'contact': ["Call me at +91 98765-43210 or email jane.doe@example.com", np.nan, "123-456-7890 info@site.org"]})

def find_contact_details(text: str) -> tuple:
    # Handle missing or NaN values
    if pd.isna(text):
        return (np.nan, np.nan)

    # --- Phone Number Pattern ---
    phone_number_pattern = re.compile(r"(\+?\d{1,3}[\s-]?)?(\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4})")
    phone_matches = re.findall(phone_number_pattern, text)

    # Extract phone number
    if len(phone_matches) == 0:
        phone_number = np.nan
    else:
        country, number = phone_matches[0]
        phone_number = (country + number).strip()

    # Remove phone number from text to isolate email
    text_without_phone = re.sub(phone_number_pattern, "", text).strip()

    # --- Email Pattern ---
    email_pattern = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
    email_match = re.search(email_pattern, text_without_phone)
    email = email_match.group() if email_match else np.nan

    return (phone_number, email)

# Apply to DataFrame
patients_df[["phone", "email"]] = patients_df["contact"].apply(find_contact_details).apply(pd.Series)
patients_df

C:\Users\Shreya Sharma\AppData\Local\Temp\ipykernel_53668\2147781388.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  patients_df["zip_code"].fillna("0.0",inplace=True)


,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,951-719-9170ZoeWellish@superrito.com,1976-07-10,121.7,66,19.6,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,1967-04-03,118.8,66,19.2,+1 (217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,402-363-6804JaeMDebord@gustr.com,1980-02-19,177.8,71,24.8,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,1951-07-26,220.9,70,31.7,+1 (732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303,United States,334-515-7487TimNeudorf@cuvox.de,1928-02-18,192.3,27,26.1,334-515-7487,TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852,United States,207-477-0579MustafaLindstrom@jourrapide.com,1959-04-10,181.1,72,24.6,207-477-0579,MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341,United States,928-284-4492RumanBisliev@gustr.com,1948-03-26,239.6,70,34.4,928-284-4492,RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110,United States,816-223-6007JinkedeKeizer@teleworm.us,1971-01-13,171.2,67,26.8,816-223-6007,JinkedeKeizer@teleworm.us
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,1952-02-13,176.9,67,27.7,1 360 443 2060,ChidaluOnyekaozulu@jourrapide.com


In [359]:
treatments_df=pd.concat((treatments_df,treatment_cut_df))
treatments_df["hba1c_change"]=treatments_df["hba1c_start"]-treatments_df["hba1c_end"]
treatments_df["given_name"]=treatments_df["given_name"].str.title()
treatments_df["surname"]=treatments_df["surname"].str.title()
treatments_df=treatments_df.melt(id_vars=["given_name","surname","hba1c_start","hba1c_end","hba1c_change"],var_name="type",value_name="dosage")
treatments_df["dosage_start"]=treatments_df["dosage"].str.split("-").str.get(0)
treatments_df["dosage_end"]=treatments_df["dosage"].str.split("-").str.get(1)

In [375]:
treatments_df["dosage_start"]=treatments_df["dosage_start"].str.replace("u","")
treatments_df["dosage_end"]=treatments_df["dosage_end"].str.replace("u","")
treatments_df=treatments_df.dropna(how="any")
treatments_df=treatments_df.merge(adverse_reactions,how="left")